In [45]:
import requests
import csv
import pandas as pd
import json
import dateutil
import time
import json

rest_api_domain = "https://api-fxpractice.oanda.com"
access_token = "";
account_id = "101-004-7361006-006"
granularity = "D"
instrument = "GBP_USD"
number_of_candles = "180"
candles_endpoint = f'/v3/instruments/{instrument}/candles?count={number_of_candles}&granularity={granularity}'
authorization_header = f'Bearer {access_token}'
trades_endpoint = f"/v3/accounts/{account_id}/trades"

In [16]:
url = rest_api_domain + trades_endpoint
response = requests.get(url, headers={'Authorization': authorization_header})
response_json = response.json()
trades = response_json['trades']

In [41]:
def datetime_to_utc_timestamp(date_time):
    return int(time.mktime(date_time.timetuple()))

In [55]:
def extract_candlesticks(response_json):
    candlesticks = []
    for candle in response_json['candles']:
        p_open = float(candle['mid']['o'])
        p_close = float(candle['mid']['c'])
        p_low = float(candle['mid']['l'])
        p_high = float(candle['mid']['h'])
        the_time = candle['time']
        parsed_date_time = dateutil.parser.parse(the_time)
        date_time_utc = datetime_to_utc_timestamp(parsed_date_time)
        candlestick = { 'open': p_open, 'close': p_close, 'low': p_low, 'high': p_high, 'time': date_time_utc}
        candlesticks.append(candlestick)
    return {'instrument': response_json['instrument'], 'granularity': response_json['granularity'], 'candles': candlesticks}
    

In [56]:
candles_url = rest_api_domain + candles_endpoint
response = requests.get(candles_url, headers={'Authorization': authorization_header})
response_json = response.json()
r = extract_candlesticks(response_json)
r

{'instrument': 'GBP_USD',
 'granularity': 'D',
 'candles': [{'open': 1.34167,
   'close': 1.33807,
   'low': 1.33446,
   'high': 1.34417,
   'time': 1528664400},
  {'open': 1.33807,
   'close': 1.33695,
   'low': 1.33419,
   'high': 1.34248,
   'time': 1528750800},
  {'open': 1.33738,
   'close': 1.33782,
   'low': 1.33082,
   'high': 1.33894,
   'time': 1528837200},
  {'open': 1.33696,
   'close': 1.32626,
   'low': 1.32564,
   'high': 1.3447,
   'time': 1528923600},
  {'open': 1.32626,
   'close': 1.3278,
   'low': 1.32118,
   'high': 1.32987,
   'time': 1529010000},
  {'open': 1.32663,
   'close': 1.32447,
   'low': 1.32262,
   'high': 1.32844,
   'time': 1529269200},
  {'open': 1.32484,
   'close': 1.31758,
   'low': 1.31507,
   'high': 1.32732,
   'time': 1529355600},
  {'open': 1.31793,
   'close': 1.31684,
   'low': 1.31454,
   'high': 1.3218,
   'time': 1529442000},
  {'open': 1.31757,
   'close': 1.32437,
   'low': 1.3102,
   'high': 1.32696,
   'time': 1529528400},
  {'open':

In [49]:
def write_to_json(json_data_as_dict):
    with open('/Users/david/Desktop/json_data.json', 'w') as fp:
        json.dump(json_data_as_dict, fp)

In [57]:
write_to_json(r)

In [17]:
trade = trades[0]
instrument = trade['instrument']
open_time = trade['openTime']
units = trade['initialUnits']
financing = trade['financing']
realized_pl = trade['realizedPL']
unrealized_pl = trade['unrealizedPL']
initial_margin_req = trade['initialMarginRequired']
margin_used = trade['marginUsed']
open_price = trade['price']
take_profit = None
stop_loss = 'Not Set'
tuple_header = "Instrument", 'Open Time', "Open Price", "Units", "P/L", "TP", "SL"
if 'takeProfitOrder' in trade:
    take_profit = trade['takeProfitOrder']['price']
if 'stopLossOrder' in trade:
    stop_loss = trade['stopLossOrder']['price']

trade_tuple = instrument, open_time, float(open_price), int(units), float(financing), float(take_profit), stop_loss
trade_tuple
tuple_header

('Instrument', 'Open Time', 'Open Price', 'Units', 'P/L', 'TP', 'SL')

In [18]:
def read_as_pandas_df(trades_json):
    df = pd.read_json(json.dumps(trades))
    return df

In [19]:
trade = trades[0]
def parse_trade_from_response(trade):
    instrument = trade['instrument']
    open_time = trade['openTime']
    units = int(trade['initialUnits'])
    direction = "Buy" if units > 0 else "Sell"
    financing = trade['financing']
    realized_pl = trade['realizedPL']
    unrealized_pl = trade['unrealizedPL']
    initial_margin_req = trade['initialMarginRequired']
    margin_used = trade['marginUsed']
    open_price = trade['price']
    take_profit = None
    stop_loss = 'Not Set'
    if 'takeProfitOrder' in trade:
        take_profit = trade['takeProfitOrder']['price']
    if 'stopLossOrder' in trade:
        stop_loss = trade['stopLossOrder']['price']

    trade_list = [instrument, open_time, float(open_price), direction, units, unrealized_pl, float(take_profit), stop_loss, float(financing)]
    return trade_list

In [20]:
def get_csv_header():
    return [str("Instrument"), str('Open Time'), "Open Price", "Direction","Units", "P/L", "TP", "SL", "Financing"]

In [21]:
def write_csv(filename,rows_as_lists):
    with open(filename, 'w') as csvfile:
        spamwriter = csv.writer(csvfile,delimiter=',')
        for row in rows_as_lists:
            spamwriter.writerow(row)

In [22]:
all_trades = [ parse_trade_from_response(t) for t in trades]

In [23]:
all_trades = [get_csv_header()] + all_trades

In [24]:
all_trades

[['Instrument',
  'Open Time',
  'Open Price',
  'Direction',
  'Units',
  'P/L',
  'TP',
  'SL',
  'Financing'],
 ['USD_MXN',
  '2019-02-18T08:59:02.184394258Z',
  19.21681,
  'Buy',
  1100,
  '1.4571',
  19.37,
  'Not Set',
  -0.0768],
 ['EUR_JPY',
  '2019-02-14T20:39:36.887297483Z',
  124.858,
  'Buy',
  1000,
  '1.5530',
  125.3,
  'Not Set',
  -0.1027],
 ['NZD_USD',
  '2019-02-14T20:34:50.191431987Z',
  0.68385,
  'Sell',
  -1100,
  '-1.0299',
  0.67,
  'Not Set',
  -0.0625],
 ['USD_MXN',
  '2019-02-14T15:30:04.495892062Z',
  19.3767,
  'Buy',
  1100,
  '-5.6115',
  19.58,
  'Not Set',
  -0.6582],
 ['USD_JPY',
  '2019-02-14T15:29:31.845291935Z',
  110.732,
  'Buy',
  1200,
  '-1.2762',
  112.0,
  'Not Set',
  0.2296],
 ['AUD_JPY',
  '2019-02-14T09:06:42.125737851Z',
  78.881,
  'Buy',
  1200,
  '-0.5709',
  79.2,
  'Not Set',
  0.0745],
 ['EUR_USD',
  '2019-02-13T14:27:26.096362670Z',
  1.12872,
  'Buy',
  1900,
  '3.4543',
  1.1345,
  'Not Set',
  -0.8906],
 ['USD_JPY',
  '2019-0

In [25]:
write_csv("/Users/david/Desktop/trades.csv",all_trades)

In [26]:
# https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
df = read_as_pandas_df(trades)
#df['Direction'] = "Buy" if int(df['currentUnits']) > 0 else "Sell"
df['Direction'] = df.apply(lambda row: "Buy" if int(row['currentUnits']) > 0 else "Sell",axis=1)
df['currentUnits'] = df.apply(lambda row: -1*int(row['currentUnits']) if int(row['currentUnits']) < 0 else int(row['currentUnits']),axis=1)
#df['openTime'] = df['openTime'].apply(dateutil.parser.parse, dayfirst=True)
buys = df['currentUnits'][df['Direction'] == 'Buy'].sum()
sells = df['currentUnits'][df['Direction'] == 'Sell'].sum()
direction_counts = df['Direction'].value_counts()
print("Directions:")
print(direction_counts)
print("=================")
print(f"Units bought: {buys}")
print(f"Units sold: {sells}")



Directions:
Buy     28
Sell    20
Name: Direction, dtype: int64
Units bought: 37920
Units sold: 21503


In [27]:
df_grouped = df.groupby(['instrument','Direction'],as_index=False)
df2 = df_grouped.agg({'currentUnits':'sum', 'financing': 'sum'})
df2['perc'] = df2.apply(lambda row: 100*row['currentUnits']/buys if row['Direction'] == 'Buy' else 100*row['currentUnits']/sells,axis=1)
df2


,instrument,Direction,currentUnits,financing,perc
0,AUD_JPY,Buy,5500,2.2241,14.504219
1,AUD_USD,Buy,10400,-12.7562,27.426160
2,BCO_USD,Buy,20,-10.2669,0.052743
3,BCO_USD,Sell,3,-0.0546,0.013952
4,EUR_GBP,Sell,1000,0.1262,4.650514
5,EUR_JPY,Buy,1000,-0.1027,2.637131
6,EUR_USD,Buy,8700,-8.1877,22.943038
7,GBP_USD,Buy,2700,-2.7377,7.120253
8,GBP_USD,Sell,1200,1.8270,5.580617
9,NZD_USD,Buy,3500,-1.3437,9.229958
